# 1. Getting Started

## 1.1 Loading a Model and Inspecting It

To begin with, masspy incorporates metabolites, reactions, initial conditions for a given metabolite and rate equations for each reaction in order to create models for simulations of cell reactions. To load a test model, type: 

In [2]:
from mass import MassReaction, MassMetabolite, MassModel
from mass.test import create_test_model
#Incorporates RBC Core Model as a test model
model= create_test_model("textbook")

The reactions, metabolites and initial contditions are attributes of the masspy model which corresponds to a special type of list called a masspy.DictList, and each one is made up of mass.Reaction and mass.Metabolite and many more respectively.

<font color='red'>Should be cobra.DictList</font>

In [3]:
print(len(model.reactions))
print(len(model.metabolites))

53
48


When using Jupyter Notebook, this type of information is rendered in a table. 

<font color='red'>iPython/Jupyter</font>

In [4]:
model

Name,Core_RBC_Model
Memory address,0x0126755f98
Stoichiometric Matrix,48x53
Matrix Rank,44
Matrix Type,"dense, float64"
Number of Metabolites,48
Number of Reactions,53
Number of Initial Conditions,48
Number of Forward Rate Constants,53
Number of Equilibrium Constants,53
Number of Irreversible Reactions,11


Just like a regular list, objects in the DictList can be retrieved by indexing. For example, to get the 30th reaction in the model (at index 29 bc of **0-indexing**)

<font color='red'>(bc --> because)</font>

In [5]:
model.reactions[29]

Reaction identifier,DPGM
Name,Diphosphoglyceromutase
Memory address,0x012679bc88
Subsystem,Hemoglobin
Stoichiometry,"_13dpg_c <=> _23dpg_c + h_c 3-Phospho-D-glyceroyl phosphate <=> 2,3-Disphospho-D-glycerate + H+"
GPR,
Kinetic Reversibility,True


Additionally, items can be retrieved by their id using the DictList.get_by_id() function. For example, to get the cytosolic atp metabolite object in our RBC core model(the id is "atp_c"), we can do the following:

In [6]:
model.metabolites.get_by_id("atp_c")

MassMetabolite identifier,atp_c
Name,ATP
Memory address,0x0126760860
Formula,C10H12N5O13P3
Compartment,c
Initial Condition,1.6
Gibbs formation energy,None
In 8 reaction(s),"PYK, ADK1, PRPPS, HEX1, ATPM, ADNK1, PGK, PFK"


Users with an interactive shell such a iPython will be able to tab complete list elements inside a list. While this is not recommended behavior for most code because of the possibility of characters like "-" inside ids, this is very useful while in an interactive prompt:

In [7]:
print(model.reactions.EX_lac__L_e.forward_rate_constant)
print(model.reactions.EX_lac__L_e.equilibrium_constant)

#put in forward and reverse rate constants

5.789814814814814
1


To see all the available attributes within the model being out object, we can do:  

<font color='red'>Define an attributes function</font>

In [9]:
def attributes(obj):
    pass
    
attributes(obj)

AttributeError: 'MassModel' object has no attribute 'attributes'

In [7]:
dir(MassReaction)

['Keq',
 'Keq_str',
 'S',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_associate_gene',
 '_dissociate_gene',
 '_repr_html_',
 '_set_id_with_model',
 '_update_awareness',
 'add_metabolites',
 'all_parameter_ids',
 'build_reaction_from_string',
 'build_reaction_string',
 'check_mass_balance',
 'compartments',
 'copy',
 'equilibrium_constant',
 'exchange',
 'external_metabolite',
 'flux_symbol',
 'forward_rate_constant',
 'functional',
 'gene_name_reaction_rule',
 'gene_reaction_rule',
 'genes',
 'get_coefficient',
 'get_coefficie

To inspect the MassModel objects, various methods exist as outline above; to highlight a few, 

In [8]:
model.reactions

[<MassReaction HEX1 at 0x127778c88>,
 <MassReaction PGI at 0x127778fd0>,
 <MassReaction PFK at 0x127788198>,
 <MassReaction FBA at 0x127788518>,
 <MassReaction TPI at 0x127788748>,
 <MassReaction GAPD at 0x1277888d0>,
 <MassReaction PGK at 0x127788cf8>,
 <MassReaction PGM at 0x127788fd0>,
 <MassReaction ENO at 0x127791198>,
 <MassReaction PYK at 0x1277913c8>,
 <MassReaction LDH_L at 0x127791748>,
 <MassReaction G6PDH2r at 0x127791ac8>,
 <MassReaction PGL at 0x127791e48>,
 <MassReaction GND at 0x127798198>,
 <MassReaction RPE at 0x1277984e0>,
 <MassReaction RPI at 0x127798668>,
 <MassReaction TKT1 at 0x1277987f0>,
 <MassReaction TKT2 at 0x127798b00>,
 <MassReaction TALA at 0x127798dd8>,
 <MassReaction ADNK1 at 0x1277a10f0>,
 <MassReaction NTD7 at 0x1277a1438>,
 <MassReaction ADA at 0x1277a1748>,
 <MassReaction AMPDA at 0x1277a1a20>,
 <MassReaction NTD11 at 0x1277a1cf8>,
 <MassReaction PUNP5 at 0x1277a1fd0>,
 <MassReaction PPM at 0x1277a82e8>,
 <MassReaction PRPPS at 0x1277a8438>,
 <Mass

As part of the DictList function, we can view all of the reactions if we so choose to; this would require us to not index the particular reaction and list out all reactions if necessary. 

In [9]:
model.metabolites

[<MassMetabolite glc__D_c at 0x127778208>,
 <MassMetabolite g6p_c at 0x127778240>,
 <MassMetabolite f6p_c at 0x127778278>,
 <MassMetabolite fdp_c at 0x1277782b0>,
 <MassMetabolite dhap_c at 0x1277782e8>,
 <MassMetabolite g3p_c at 0x127778320>,
 <MassMetabolite _13dpg_c at 0x127778358>,
 <MassMetabolite _3pg_c at 0x127778390>,
 <MassMetabolite _2pg_c at 0x1277783c8>,
 <MassMetabolite pep_c at 0x127778400>,
 <MassMetabolite pyr_c at 0x127778438>,
 <MassMetabolite lac__L_c at 0x127778470>,
 <MassMetabolite _6pgl_c at 0x1277784a8>,
 <MassMetabolite _6pgc_c at 0x1277784e0>,
 <MassMetabolite ru5p__D_c at 0x127778518>,
 <MassMetabolite xu5p__D_c at 0x127778550>,
 <MassMetabolite r5p_c at 0x127778588>,
 <MassMetabolite s7p_c at 0x1277785c0>,
 <MassMetabolite e4p_c at 0x1277785f8>,
 <MassMetabolite ade_c at 0x127778630>,
 <MassMetabolite adn_c at 0x127778668>,
 <MassMetabolite imp_c at 0x1277786a0>,
 <MassMetabolite ins_c at 0x1277786d8>,
 <MassMetabolite hxan_c at 0x127778710>,
 <MassMetabolit

To examine the stoichiometric matrix, we can do,

In [10]:
model.S

array([[-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1., -1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

This would be better viewed through the pandas.DataFrame table; to do so, 

<font color='red'>Show how to increase max rows/columns, and format with pd.options</font>

In [11]:
model.update_S(matrix_type="DataFrame", update_model=True)

,HEX1,PGI,PFK,FBA,TPI,GAPD,PGK,PGM,ENO,PYK,...,EX_ade_e,EX_adn_e,EX_ins_e,EX_hxan_e,EX_pi_e,EX_h_e,EX_h2o_e,EX_co2_e,EX_nh3_e,EX_o2_e
glc__D_c,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g6p_c,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f6p_c,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fdp_c,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dhap_c,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3p_c,0.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_13dpg_c,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_3pg_c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_2pg_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pep_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Model rates can be examined by iterating using a "for" loop

<font color='red'>Explain what the datatype is (e.g. rates are a dict, can be iterated using items() method)</font>

In [12]:
for rxn, rate in model.rates.items():
    print("%s: %s" % (rxn.id, rate))

HEX1: kf_HEX1*(atp_c(t)*glc__D_c(t) - adp_c(t)*g6p_c(t)/Keq_HEX1)
PGI: kf_PGI*(g6p_c(t) - f6p_c(t)/Keq_PGI)
PFK: kf_PFK*(atp_c(t)*f6p_c(t) - adp_c(t)*fdp_c(t)/Keq_PFK)
FBA: kf_FBA*(fdp_c(t) - dhap_c(t)*g3p_c(t)/Keq_FBA)
TPI: kf_TPI*(dhap_c(t) - g3p_c(t)/Keq_TPI)
GAPD: kf_GAPD*(g3p_c(t)*nad_c(t)*pi_c(t) - _13dpg_c(t)*nadh_c(t)/Keq_GAPD)
PGK: kf_PGK*(_13dpg_c(t)*adp_c(t) - _3pg_c(t)*atp_c(t)/Keq_PGK)
PGM: kf_PGM*(_3pg_c(t) - _2pg_c(t)/Keq_PGM)
ENO: kf_ENO*(_2pg_c(t) - pep_c(t)/Keq_ENO)
PYK: kf_PYK*(adp_c(t)*pep_c(t) - atp_c(t)*pyr_c(t)/Keq_PYK)
LDH_L: kf_LDH_L*(nadh_c(t)*pyr_c(t) - lac__L_c(t)*nad_c(t)/Keq_LDH_L)
G6PDH2r: kf_G6PDH2r*(g6p_c(t)*nadp_c(t) - _6pgl_c(t)*nadph_c(t)/Keq_G6PDH2r)
PGL: kf_PGL*(_6pgl_c(t) - _6pgc_c(t)/Keq_PGL)
GND: kf_GND*(_6pgc_c(t)*nadp_c(t) - co2_c(t)*nadph_c(t)*ru5p__D_c(t)/Keq_GND)
RPE: kf_RPE*(ru5p__D_c(t) - xu5p__D_c(t)/Keq_RPE)
RPI: kf_RPI*(ru5p__D_c(t) - r5p_c(t)/Keq_RPI)
TKT1: kf_TKT1*(r5p_c(t)*xu5p__D_c(t) - g3p_c(t)*s7p_c(t)/Keq_TKT1)
TKT2: kf_TKT2*(e4

We can also call one rate and inspect it specifically instead of all the rates,

<font color='red'>Also show dict.get(key, None) method.
Great place to show how to use sympy.pprint()
Also explain datatype of rate and ode (e.g. sympy.Basic expression)
Turn an expression into a string to show how its done</font>


In [17]:
import sympy as sym

print(type(str(model.rates[model.reactions.HEX1])))
print(isinstance(model.rates[model.reactions.HEX1], str))
print(model.rates.get("hey bb", "Oh no"))

<class 'str'>
False
Oh no


In [14]:
model.rates

{<MassReaction HEX1 at 0x127778c88>: kf_HEX1*(atp_c(t)*glc__D_c(t) - adp_c(t)*g6p_c(t)/Keq_HEX1),
 <MassReaction PGI at 0x127778fd0>: kf_PGI*(g6p_c(t) - f6p_c(t)/Keq_PGI),
 <MassReaction PFK at 0x127788198>: kf_PFK*(atp_c(t)*f6p_c(t) - adp_c(t)*fdp_c(t)/Keq_PFK),
 <MassReaction FBA at 0x127788518>: kf_FBA*(fdp_c(t) - dhap_c(t)*g3p_c(t)/Keq_FBA),
 <MassReaction TPI at 0x127788748>: kf_TPI*(dhap_c(t) - g3p_c(t)/Keq_TPI),
 <MassReaction GAPD at 0x1277888d0>: kf_GAPD*(g3p_c(t)*nad_c(t)*pi_c(t) - _13dpg_c(t)*nadh_c(t)/Keq_GAPD),
 <MassReaction PGK at 0x127788cf8>: kf_PGK*(_13dpg_c(t)*adp_c(t) - _3pg_c(t)*atp_c(t)/Keq_PGK),
 <MassReaction PGM at 0x127788fd0>: kf_PGM*(_3pg_c(t) - _2pg_c(t)/Keq_PGM),
 <MassReaction ENO at 0x127791198>: kf_ENO*(_2pg_c(t) - pep_c(t)/Keq_ENO),
 <MassReaction PYK at 0x1277913c8>: kf_PYK*(adp_c(t)*pep_c(t) - atp_c(t)*pyr_c(t)/Keq_PYK),
 <MassReaction LDH_L at 0x127791748>: kf_LDH_L*(nadh_c(t)*pyr_c(t) - lac__L_c(t)*nad_c(t)/Keq_LDH_L),
 <MassReaction G6PDH2r at 0x1

The same goes for ODE's; the following function is not being printed for the sake of organization

In [41]:
for metabolite, ode in model.odes.items():
    print("%s:%s" % (metabolite.id,ode))

glc__D_c:glc__D_e*kf_S_glc__D_e - kf_HEX1*(atp_c(t)*glc__D_c(t) - adp_c(t)*g6p_c(t)/Keq_HEX1)
g6p_c:-kf_G6PDH2r*(g6p_c(t)*nadp_c(t) - _6pgl_c(t)*nadph_c(t)/Keq_G6PDH2r) + kf_HEX1*(atp_c(t)*glc__D_c(t) - adp_c(t)*g6p_c(t)/Keq_HEX1) - kf_PGI*(g6p_c(t) - f6p_c(t)/Keq_PGI)
f6p_c:-kf_PFK*(atp_c(t)*f6p_c(t) - adp_c(t)*fdp_c(t)/Keq_PFK) + kf_PGI*(g6p_c(t) - f6p_c(t)/Keq_PGI) + kf_TALA*(g3p_c(t)*s7p_c(t) - e4p_c(t)*f6p_c(t)/Keq_TALA) + kf_TKT2*(e4p_c(t)*xu5p__D_c(t) - f6p_c(t)*g3p_c(t)/Keq_TKT2)
fdp_c:-kf_FBA*(fdp_c(t) - dhap_c(t)*g3p_c(t)/Keq_FBA) + kf_PFK*(atp_c(t)*f6p_c(t) - adp_c(t)*fdp_c(t)/Keq_PFK)
dhap_c:kf_FBA*(fdp_c(t) - dhap_c(t)*g3p_c(t)/Keq_FBA) - kf_TPI*(dhap_c(t) - g3p_c(t)/Keq_TPI)
g3p_c:kf_FBA*(fdp_c(t) - dhap_c(t)*g3p_c(t)/Keq_FBA) - kf_GAPD*(g3p_c(t)*nad_c(t)*pi_c(t) - _13dpg_c(t)*nadh_c(t)/Keq_GAPD) - kf_TALA*(g3p_c(t)*s7p_c(t) - e4p_c(t)*f6p_c(t)/Keq_TALA) + kf_TKT1*(r5p_c(t)*xu5p__D_c(t) - g3p_c(t)*s7p_c(t)/Keq_TKT1) + kf_TKT2*(e4p_c(t)*xu5p__D_c(t) - f6p_c(t)*g3p_c(t)/Keq

Similarly, one ODE can be pulled out,

In [56]:
model.odes[model.metabolites.glc__D_c] #*metabolites don't have ODE until they are put in*

glc__D_e*kf_S_glc__D_e - kf_HEX1*(atp_c(t)*glc__D_c(t) - adp_c(t)*g6p_c(t)/Keq_HEX1)

## 1.2 Reactions

We will consider the reaction glucose-6-phosphate isomerase, which interconverts glucose-6-phosphate to fructose-6-phosphate. The reaction model for this reaction in our test model is PGI

In [16]:
pgi= model.reactions.get_by_id("PGI")
pgi

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x0127778fd0
Subsystem,Glycolysis
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,
Kinetic Reversibility,True


We can view the full name and reaction catalyzed as strings.

In [17]:
print(pgi.name)
print(pgi.reaction)

Glucose-6-phosphate isomerase
g6p_c <=> f6p_c


We can also view reaction upper and lower bounds. Bc pgi.lower_bound <0 and pgi.upper_bound> 0, pgi is considered reversible.

In [1]:
# print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
# print(pgi.reversible) 
#MASSPY : Technically everythign is reversible; talk about reversibility and refer to the bounds; talk about how rxns have reversible attributes and rev
# rxns can be initialized to  bounds for -1000 to 1000 or 0 to 1000 for rev and irrev resp; helps cobrapy to masspy users 


We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [19]:
pgi.check_mass_balance()

{}

In order to add a metabolite, we pass in a dict with the metabolite object and its coefficient.

In [20]:
pgi. add_metabolites({model.metabolites.get_by_id("h_c"):-1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

The reaction is no longer mass balanced.

In [21]:
pgi.check_mass_balance()

{'charge': -1, 'H': -1}

We can remove metabolite and the reaction can be balanced once again.

In [22]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"):-1})
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


Further inspection of the reaction itself can be accomplished through several methods listed below:

In [26]:
pgi.id

'PGI'

In [27]:
pgi.reactants

[<MassMetabolite g6p_c at 0x127778240>]

In [28]:
pgi.products

[<MassMetabolite f6p_c at 0x127778278>]

In [29]:
pgi.stoichiometry

[1, -1]

As stated above, dir(MassReaction) can pull up the documentation for MassReaction Class,

## 1.3 Metabolites 

We will consider cytosilic ATP as the metabolite and use id "atp_c" within the following test model. 

In [30]:
atp= model.metabolites.get_by_id("atp_c")
atp

MassMetabolite identifier,atp_c
Name,ATP
Memory address,0x0127778a20
Formula,C10H12N5O13P3
Compartment,c
Initial Condition,1.6
Gibbs formation energy,None
In 8 reaction(s),"PRPPS, ATPM, HEX1, ADNK1, PGK, PFK, PYK, ADK1"


We can print out the metabolite name and compartment (cytosol in this case) directly as string. 

In [31]:
print(atp.name)
print(atp.compartment)

ATP
c


We can also determine the charge of ATP within the model

In [32]:
atp.charge

-4

Chemical formula of the metabolite can be observed as well. 

In [33]:
print(atp.formula)

C10H12N5O13P3


The reactions attribute gives a frozenset of all reactions using given metabolite. We can use this to count the number of reactions which use ATP.

In [34]:
len(atp.reactions)

8

A metabolite like G6P will participate in fewer reactions.

In [35]:
model.metabolites.get_by_id("g6p_c").reactions

frozenset({<MassReaction G6PDH2r at 0x127791ac8>,
           <MassReaction HEX1 at 0x127778c88>,
           <MassReaction PGI at 0x127778fd0>})

## 1.4 Parameters & Initial Conditions

Through using MassPy, one can inspect initial conditions and parameters placed within the model.

In [39]:
model.initial_conditions

{<MassMetabolite _13dpg_c at 0x127778358>: 0.0003051,
 <MassMetabolite _23dpg_c at 0x1277787b8>: 3.89225503,
 <MassMetabolite _2pg_c at 0x1277783c8>: 0.01426657,
 <MassMetabolite _3pg_c at 0x127778390>: 0.09747345,
 <MassMetabolite _6pgc_c at 0x1277784e0>: 0.04877853,
 <MassMetabolite _6pgl_c at 0x1277784a8>: 0.00176584,
 <MassMetabolite ade_c at 0x127778630>: 0.00099988,
 <MassMetabolite adn_c at 0x127778668>: 0.00119991,
 <MassMetabolite adp_c at 0x1277789e8>: 0.22421214,
 <MassMetabolite amp_c at 0x1277789b0>: 0.067291,
 <MassMetabolite atp_c at 0x127778a20>: 1.23266821,
 <MassMetabolite co2_c at 0x127778be0>: 1.0000021,
 <MassMetabolite dhap_c at 0x1277782e8>: 0.17280752,
 <MassMetabolite dhb_c at 0x127778908>: 0.05792551,
 <MassMetabolite e4p_c at 0x1277785f8>: 0.00516196,
 <MassMetabolite f6p_c at 0x127778278>: 0.02573503,
 <MassMetabolite fdp_c at 0x1277782b0>: 0.01727449,
 <MassMetabolite g3p_c at 0x127778320>: 0.00800928,
 <MassMetabolite g6p_c at 0x127778240>: 0.06307568,
 <M

From the list above, we can inspect one specific initial condition instead of all of them: 

In [47]:
model.initial_conditions[model.metabolites._13dpg_c]
#can also get initial conditions through met.ic
#important to point out diff b/w met.ic and model.ic
#Include kf/ KEq

0.0003051

A very useful functionality within Python is the ability to determine the list of available classes within each attribute utilizing the directory.

<font color='red'>Not necessary</font>

In [48]:
dir(model.initial_conditions)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

With this, you can call out and inspect the each attribute

In [50]:
model.initial_conditions.__class__

dict

In [53]:
model.initial_conditions.__format__

<function dict.__format__(format_spec, /)>

These tools are espiecially important in determining how to inspect models; in the next workbook, we will discuss how to create models from the ground up and contextualize the methods to derive the model. 


<font color='red'>Summary of Main Points</font>